# Predicting Himalayan Expedition Success
***

## Introduction

The Himalayan Database consists of records for all expeditions in the Nepal Himalayas from 1905 through 2018. The database covers expeditions to more than 450 significant peaks, including Everest, Cho Oyu and Kangchenjunga. More information can be found at http://himalayandatabase.com/index.html.

### Datasets
Two datasets are of interest: expeditions.csv and members.csv (http://himalayandatabase.com/downloads.html).

###### peaks.csv
This file contains information about each one of the more than 450 peaks, location, region, status, first ascent and more.

###### expedions.csv
This file contains information including date, peak, route, suplemantal oxygen, group size and sherpas.

###### members.csv
This file contains biographical information on each member in the expedition, including nationality, age, oxygen use, and success.

### Outcome

Nearly 10,000 expedition records and over 70,000 members records can then predict member success of a summitting a certain peak, during a certain time of year, with certain support mechanisms.  

## Packages and setup

In [1]:
import pandas as pd
import re
from IPython.core.interactiveshell import InteractiveShell

In [2]:
# Set workspace

# Set output charackters to 110 (not 79)
pd.options.display.width = 110
# To give multiple cell output. Not just the last command.
InteractiveShell.ast_node_interactivity = "all"

## Peaks dataset cleaning

In [4]:
# Load peaks as 'peaks_csv'
peaks_csv = pd.read_csv('peaks.csv')
print('Shape:', peaks_csv.shape)

peaks_csv.info()

Shape: (468, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 22 columns):
peakid        468 non-null object
pkname        468 non-null object
pkname2       238 non-null object
location      467 non-null object
heightm       468 non-null int64
heightf       468 non-null int64
himal         468 non-null int64
region        468 non-null int64
open          468 non-null bool
unlisted      468 non-null bool
trekking      468 non-null bool
trekyear      29 non-null float64
restrict      270 non-null object
phost         468 non-null int64
pstatus       468 non-null int64
pyear         325 non-null float64
pseason       468 non-null int64
pexpid        322 non-null object
psmtdate      319 non-null object
pcountry      325 non-null object
psummiters    460 non-null object
psmtnote      75 non-null object
dtypes: bool(3), float64(2), int64(7), object(10)
memory usage: 70.9+ KB


Referring to the 'Himalayan Database Guide.pdf', the following features will not be included:

pkname, pkname2 - The names of the peaks are not required for analysis, but can be inluded post-analysis.

location - This field gives a long description of the expedition only. The locations are better captured with categorical data 'himal' and 'region'.

heightf - Only 'heightm', the height in meters, are used.

trekking, trekyear - There are only 29 non-null values. Not enough to make it significant.

pseason, pexpid, psmtdate, pcountry, psummiters, psmnote - Information on the first ascent of a certain peak. Not relevant.

In [105]:
# Select only relevant peaks features 
peaks_features = ['peakid', 'heightm', 'himal', 'region','open','unlisted', 'restrict', 'phost', 'pstatus', 'pyear']

# New dataframe with only selected columns
peaks = peaks_csv[peaks_features]

# Inspect the data
peaks.head()
peaks.info()
peaks.describe()

,peakid,heightm,himal,region,open,unlisted,trekking,trekyear,restrict,phost,pstatus,pyear
0,ACHN,6055,17,7,True,False,False,NaN,Opened in 2014,1,2,2015.0
1,AGLE,6675,19,2,True,False,False,NaN,Opened in 2014,1,1,NaN
2,AMAD,6814,12,2,True,False,False,NaN,NaN,1,2,1961.0
3,AMOT,6393,3,5,True,False,False,NaN,Opened in 2002,1,1,NaN
4,AMPG,5630,12,2,True,False,False,NaN,Opened in 2002,1,2,1953.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 12 columns):
peakid      468 non-null object
heightm     468 non-null int64
himal       468 non-null int64
region      468 non-null int64
open        468 non-null bool
unlisted    468 non-null bool
trekking    468 non-null bool
trekyear    29 non-null float64
restrict    270 non-null object
phost       468 non-null int64
pstatus     468 non-null int64
pyear       325 non-null float64
dtypes: bool(3), float64(2), int64(5), object(2)
memory usage: 34.4+ KB


,heightm,himal,region,trekyear,phost,pstatus,pyear
count,468.000000,468.000000,468.000000,29.000000,468.000000,468.000000,325.000000
mean,6656.653846,10.258547,3.820513,1918.758621,2.126068,1.707265,1983.510769
std,571.898154,5.585948,2.150107,365.364168,1.514782,0.455505,23.674178
min,5407.000000,1.000000,1.000000,20.000000,1.000000,1.000000,1909.000000
25%,6235.750000,5.000000,2.000000,1978.000000,1.000000,1.000000,1963.000000
50%,6559.500000,11.000000,4.000000,1978.000000,1.000000,2.000000,1981.000000
75%,6911.000000,15.000000,6.000000,2002.000000,4.000000,2.000000,2007.000000
max,8850.000000,20.000000,7.000000,2002.000000,6.000000,2.000000,2018.000000


The missing values for 'restrict' will be negated when introducing 'expeditions.csv'. Example, a peak's open status ('open' = True) changes from False on year 'openyear', when the Nepal government approved the peak.

In the same way 'restrict' indicates the year the 'open' status changed between closed or open, however in this case the year and open/closed inside the 'restrict' column will have to be used to create a new column indicating when the peak's status changed. If the new column, called 'openyear' contains a date (year), the 'open' value will simply switch between True/False.

The missing values for column 'pyear', or the first ascent year, will be determined from 'expedition.csv' where the fist expedition date for the specific peak took place.

The column 'trekyear' is type float due to the NaN values. Notice 'trekyear' has a minimum value of 20, which can't be right.

In [6]:
# Inspect 'trekyear'
peaks['trekyear'].value_counts(dropna=False)

NaN       439
1978.0     18
2002.0     10
20.0        1
Name: trekyear, dtype: int64

Referring to the one entry with year '20', if you looking at the value counts, the only other option for an entry is the year 1978, but the entry already starts with '20'. Therefore assume the entry should have been the year 2002.

In any case, with only 2 years for which these entries exits, 1978 and 2002, one cannot assume that the features, 'trekking' and 'trekyear' will affect a significant difference. In addition, the Nepalese government does not include trekking peaks in their statistical reports and analysis. Therefore, 'trekking' and 'trekyear' will be ommited.

In [7]:
# Deleting columns 'trekking' and 'trekyear'.
peaks = peaks.drop(['trekking', 'trekyear'], axis=1, errors='ignore')
peaks.columns

Index(['peakid', 'heightm', 'himal', 'region', 'open', 'unlisted', 'restrict', 'phost', 'pstatus', 'pyear'], dtype='object')

In [8]:
# Inspect 'open' and 'restrict'

# Exclude all NaN's and retain DataFrame
inspect = peaks[peaks['restrict'].notnull()][['open', 'restrict']]
# Only select rows that contain values in 'restrict' 
inspect_open = inspect[inspect['restrict'].str.contains('Open')]

# Create pattern that identifies "Opened in <year>" from column 'restrict'.
pattern = re.compile('Opened in \d{4}$')
mask = inspect_open['restrict'].apply(lambda x : True if bool(pattern.match(x)) else False)
inspect_open[mask].head()

# Now only show the rows that don't match the pattern "Opened in <year>" for further inspection.
inspect_open[~mask]

,open,restrict
0,True,Opened in 2014
1,True,Opened in 2014
3,True,Opened in 2002
4,True,Opened in 2002
5,True,Opened in 2002


,open,restrict
40,False,Opened in 2002 as trekking peak
53,False,Opened in 1997; converted to trekking peak in ...
142,True,Opened in 2013?
146,False,"Opened in 2002, delisted in 2013?"
219,False,Opened in 2002 as trekking peak
301,True,Opened in 2014 as Hillary Peak
302,True,Opened in 2014 as Tenzing Peak
310,False,Opened in 2002 as trekking peak
319,False,Opened in 2002 as trekking peak
377,True,Opened in 2014 (as Khang Karpo)


As can be seen, where 'open' = False, the 'restrict' value only refers to trekking status, thus those values can set to NaN.

In [9]:
# Find indices where 'open' = False AND 'restrict' does not match the pattern
val_to_remove = inspect_open[~mask]
#val_to_remove[val_to_remove['open'] == False]

# Find only these calues in peaks DataFrame
idx = val_to_remove[val_to_remove['open'] == False].index
#peaks.loc[idx]

# Change these values to NaN
peaks['restrict'][idx] = None

# Check new correct values
peaks.loc[idx]

# Look for the other 'Opened in ...' entries and 
peaks['restrict'].value_counts(dropna=False)

C:\Users\jacqu\Miniconda3\envs\springboard_env\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,peakid,heightm,himal,region,open,unlisted,restrict,phost,pstatus,pyear
40,CHEK,6121,19,2,False,False,None,4,2,2005.0
53,CHRI,5550,12,2,False,False,None,1,2,1965.0
146,HUNK,6119,12,2,False,True,None,1,2,2012.0
219,KYAZ,6151,12,2,False,False,None,1,2,2002.0
310,NREK,6159,12,2,False,False,None,1,2,NaN
319,OMBG,6340,12,2,False,False,None,1,2,1960.0


NaN                                                   204
Opened in 2014                                         97
Opened in 2002                                         92
Opened in 2003                                         40
Opened in 2001                                          4
Opened in 1997                                          4
Converted to trekking peak in 2002                      3
One permit for Churen Himal's three summits             3
Requires permit for Jobo Rinjang                        3
Peak entirely within China                              2
Requires permit for Nuptse                              2
Requires permit for Annapurna I                         2
Closed                                                  1
One permit for Kanjiroba's north and south summits      1
Requires permit for Amphu I                             1
Opened in 2014 (as Khang Karpo)                         1
Opened in 2014 as Tenzing Peak                          1
Requires permi

In [10]:
# Now the year values can be retreived and saved as float in a new column.
# Why not int? Pandas float works better when series contains NaN's

# Inspect 'open' and 'restrict' again after some values changed in 'restrict'
# Exclude all NaN's and retain DataFrame
inspect = peaks[peaks['restrict'].notnull()][['open', 'restrict']]
# Only select rows that contain values in 'restrict' 
inspect_open = inspect[inspect['restrict'].str.contains('Open')]

# New values
inspect_open['restrict'].value_counts()

# Get year information only
pattern2 = re.compile('\d{4}')

# Add peaks column 'openyear' and remove column 'restrict'
peaks['openyear'] = inspect_open['restrict'].apply(lambda x : re.findall(pattern2, x)[0])
peaks = peaks.drop('restrict', axis=1, errors='ignore')

Opened in 2014                     97
Opened in 2002                     92
Opened in 2003                     40
Opened in 2001                      4
Opened in 1997                      4
Opened in 2014 (as Khang Karpo)     1
Opened in 2013?                     1
Opened in 2014 as Tenzing Peak      1
Opened in 2016                      1
Opened in 2014 as Hillary Peak      1
Name: restrict, dtype: int64

In [11]:
# Convert columns 'openyear' to float
peaks['openyear'] = peaks['openyear'].astype(float)

# Review data
peaks.head()
peaks.info()

,peakid,heightm,himal,region,open,unlisted,phost,pstatus,pyear,openyear
0,ACHN,6055,17,7,True,False,1,2,2015.0,2014.0
1,AGLE,6675,19,2,True,False,1,1,NaN,2014.0
2,AMAD,6814,12,2,True,False,1,2,1961.0,NaN
3,AMOT,6393,3,5,True,False,1,1,NaN,2002.0
4,AMPG,5630,12,2,True,False,1,2,1953.0,2002.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 10 columns):
peakid      468 non-null object
heightm     468 non-null int64
himal       468 non-null int64
region      468 non-null int64
open        468 non-null bool
unlisted    468 non-null bool
phost       468 non-null int64
pstatus     468 non-null int64
pyear       325 non-null float64
openyear    242 non-null float64
dtypes: bool(2), float64(2), int64(5), object(1)
memory usage: 30.2+ KB


In [12]:
# Lastly, convert himal, region, phost and pstatus to category
peaks[['himal', 'region', 'phost', 'pstatus']] = peaks[['himal', 'region', 'phost', 'pstatus']].astype('category')
peaks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 10 columns):
peakid      468 non-null object
heightm     468 non-null int64
himal       468 non-null category
region      468 non-null category
open        468 non-null bool
unlisted    468 non-null bool
phost       468 non-null category
pstatus     468 non-null category
pyear       325 non-null float64
openyear    242 non-null float64
dtypes: bool(2), category(4), float64(2), int64(1), object(1)
memory usage: 18.9+ KB


The 'peaks' data set is complete. Columns with NaN values 'pyear' and 'openyear' will be completed when joined with the 'epedition' data set.

## Expedition dataset cleaning

In [488]:
# Load expeditions as 'xpd' for short
xpd = pd.read_csv('expeditions.csv')

# Inspect the total entries and non-null values
xpd_csv.shape
xpd_csv.info()

(9959, 65)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9959 entries, 0 to 9958
Data columns (total 65 columns):
expid         9959 non-null object
peakid        9959 non-null object
year          9959 non-null int64
season        9959 non-null int64
host          9959 non-null int64
route1        9806 non-null object
route2        307 non-null object
route3        30 non-null object
route4        5 non-null object
nation        9959 non-null object
leaders       9935 non-null object
sponsor       9139 non-null object
success1      9959 non-null bool
success2      9959 non-null bool
success3      9959 non-null bool
success4      9959 non-null bool
ascent1       2719 non-null object
ascent2       101 non-null object
ascent3       11 non-null object
ascent4       4 non-null object
claimed       9959 non-null bool
disputed      9959 non-null bool
countries     3224 non-null object
approach      4677 non-null object
bcdate        9959 non-null object
smtdate       9959 non-null object
smttime   

# Referring to the 'Himalayan Database Guide.pdf', the following features will not be included:

route2-4 - Since 'route1' has significantly less null values compared to 'route2/3/4', only 'route1' will be used. There is also little information on 'route2/3/4' information in the documentation.

sponsor - Looking at it's value_counts, there are too many NaN values and sponsors with the most counts are independent, non-descript entries.

success2-4 - Same as 'route2-4' description above.

ascent1-4 - For the same reason as 'route2-4', 'ascent1-4' has not contain enough data and according to the documentation, these numbers are not maintained for recent ascends of some peaks.

claimed - This describes disputed successes which, according to the documentation, is not included as successes in statistical reports and analysis. Value_counts only give 30 True values, which does not affect 'success1'.

disputed - This describes the disputed nature of this summit, but does not affect the outcome of 'success1'.

countries - Too many NaN values. This data will be captured by the members dataset.

approach - Too many NaN values.

bcdate - The base camp date, 'bcdate', will be discounted by the summit date, as more relevant.

smttime, smtdays, totdays, termdate - Too many Null values.

termnote, highpoint - Not relevant.

o2unkwn - Only 21 True values. The other oxygen related columns covers the 21 unknown oxygen use columns.

othersmts, campsites, accidents, achievement - Detailed descriptive information. Cannot be used.

stdrte - This is the 8000m standard route. Not applicable to all peaks then.

primrte, primmem, primref, primid, chksum - Not relevent. Database related fields.


Output (Y) - Feature 'termreason' will be used as the outcome.

In [489]:
# Only using the following features
drop_columns = ['route2', 'route3', 'route4', 'sponsor', 'success2', 'success3', 'success4', 'ascent1', 
                'ascent2', 'ascent3', 'ascent4', 'claimed', 'disputed', 'countries', 'approach', 'bcdate', 
                'smttime', 'smtdays', 'totdays', 'termdate', 'termnote', 'highpoint', 'o2unkwn', 'othersmts', 'campsites', 
                'accidents', 'achievment', 'stdrte', 'primrte', 'primmem', 'primref', 'primid', 'chksum']

# Drop the columns not needed
xpd.drop(drop_columns, inplace=True, axis=1)

xpd.shape
xpd.columns

(9959, 32)

Index(['expid', 'peakid', 'year', 'season', 'host', 'route1', 'nation', 'leaders', 'success1', 'smtdate',
       'termreason', 'traverse', 'ski', 'parapente', 'camps', 'rope', 'totmembers', 'smtmembers', 'mdeaths',
       'tothired', 'smthired', 'hdeaths', 'nohired', 'o2used', 'o2none', 'o2climb', 'o2descent', 'o2sleep',
       'o2medical', 'o2taken', 'agency', 'comrte'],
      dtype='object')

Further data cleaning follows.

Note: For 'nation' former names of countries have to be used. Example, expeditions from Eastern Bloc countries before 1993 were recorded as USSR. It is not possible now to tell from which post-1993 country they belong to. Therefore post-1993 expeditions for these countries will be grouped under USSR. 

Czech Republic and Slovakia will change to Czechoslovakia.
Belarus, Estonia, Georgia, Kazakhstan, Latvia, Lithuania, Russia, Ukrain and Uzbekistan will change to USSR.
Exception; 'W Germany' will change to 'Germany'. 

In [490]:
# Clean the data for the remaining features

# Convert these columns to category
to_cat = ['season', 'host', 'termreason']
for col in to_cat:
    xpd[col] = xpd[col].astype('category')

# Convert these columns from bool to binary
to_bin = ['success1','traverse', 'ski', 'parapente', 'nohired', 'o2used', 'o2none', 'o2climb', 'o2descent', 
          'o2sleep', 'o2medical', 'o2taken', 'comrte']
xpd[to_bin] = xpd[to_bin].astype(int)

# 'route1' has 153 NaN. Change each NaN value to the index number as a random name. 
def replace_nan_with_diff_object_val(column):
    """ Replace NaN values in a column with different object values"""
    nan_range = xpd[column][xpd[column].isnull()].index
    for i in nan_range:
        xpd[column][i] = i

replace_nan_with_diff_object_val('route1')
        
# 'nation' includes  older names of countries in the previous Eastern Bloc. For consistency, change 
# new country names to older ones. 
xpd['nation'] = xpd['nation'].replace(to_replace='W Germany/Iran', value='Germany/Iran')
xpd['nation'] = xpd['nation'].replace(to_replace='W Germany', value='Germany')

current_name = ['Czech Republic', 'Slovakia']
for name in current_name:
    xpd['nation'] = xpd['nation'].replace(to_replace=name, value='Czechoslovakia')
    
current_name = ['Croatia', 'Macedonia', 'Serbia',' Slovenia']
for name in current_name:
    xpd['nation'] = xpd['nation'].replace(to_replace=name, value='Yugoslavia')
    
current_name = ['Belarus', 'Estonia', 'Georgia', 'Kazakhstan', 'Latvia', 'Lithuania', 'Russia', 
                'Ukrain', 'Uzbekistan']
for name in current_name:
    xpd['nation'] = xpd['nation'].replace(to_replace=name, value='USSR')

# leaders has 24 NaN. Change each NaN value to the index number as a random name.
replace_nan_with_diff_object_val('leaders')

# smtdate get month data only as category. There are 634 NaNs. 
# Extract month information
xpd['smtmonth'] = xpd['smtdate'].str.split('/').str[1]
# Convert to float in order to calcualte mean and replace NaNs with the mean
xpd['smtmonth'] = xpd['smtmonth'].astype(float)
xpd['smtmonth'] = xpd['smtmonth'].fillna(xpd['smtmonth'].mean())
# Convert back to int to drop the decimals, then convert to category
xpd['smtmonth'] = xpd['smtmonth'].astype(int).astype('category')
# Drop the 'smtdate' column - not needed.
xpd.drop('smtdate', inplace=True, axis=1)

# tothired - (nohired = True) indicates true zero for 'tothired'. Find the Falses and allocate 
# mean values. There are 260 missing values.
xpd['tothired'][idx] = xpd['tothired'].mean().round()

# agency has 1565 NaN. 
replace_nan_with_diff_object_val('agency')

# Inspect cleaned xpd dataframe
xpd.info()

C:\Users\jacqu\Miniconda3\envs\springboard_env\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jacqu\Miniconda3\envs\springboard_env\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9959 entries, 0 to 9958
Data columns (total 32 columns):
expid         9959 non-null object
peakid        9959 non-null object
year          9959 non-null int64
season        9959 non-null category
host          9959 non-null category
route1        9959 non-null object
nation        9959 non-null object
leaders       9959 non-null object
success1      9959 non-null int32
termreason    9959 non-null category
traverse      9959 non-null int32
ski           9959 non-null int32
parapente     9959 non-null int32
camps         9959 non-null int64
rope          9959 non-null int64
totmembers    9959 non-null int64
smtmembers    9959 non-null int64
mdeaths       9959 non-null int64
tothired      9959 non-null float64
smthired      9959 non-null int64
hdeaths       9959 non-null int64
nohired       9959 non-null int32
o2used        9959 non-null int32
o2none        9959 non-null int32
o2climb       9959 non-null int32
o2descent     9959 non-null

Combine the 'peaks' and 'exp' dataframes depending on the date of the expedition and the status of the peak.

In [491]:
peaks.shape
xpd.shape

peaks.head()
xpd.head()

(468, 12)

(9959, 32)

,peakid,heightm,himal,region,open,unlisted,trekking,trekyear,restrict,phost,pstatus,pyear
0,ACHN,6055,17,7,True,False,False,NaN,Opened in 2014,1,2,2015.0
1,AGLE,6675,19,2,True,False,False,NaN,Opened in 2014,1,1,NaN
2,AMAD,6814,12,2,True,False,False,NaN,NaN,1,2,1961.0
3,AMOT,6393,3,5,True,False,False,NaN,Opened in 2002,1,1,NaN
4,AMPG,5630,12,2,True,False,False,NaN,Opened in 2002,1,2,1953.0


,expid,peakid,year,season,host,route1,nation,leaders,success1,termreason,...,o2used,o2none,o2climb,o2descent,o2sleep,o2medical,o2taken,agency,comrte,smtmonth
0,ANN260101,ANN2,1960,1,1,NW Ridge-W Ridge,UK,J. O. M. Roberts,1,1,...,1,0,1,0,1,0,0,0,0,5
1,ANN269301,ANN2,1969,3,1,NW Ridge-W Ridge,Yugoslavia,Ales Kunaver,1,1,...,0,1,0,0,0,0,0,1,0,10
2,ANN273101,ANN2,1973,1,1,W Ridge-N Face,Japan,Yukio Shimamura,1,1,...,0,1,0,0,0,0,0,2,0,5
3,ANN278301,ANN2,1978,3,1,N Face-W Ridge,UK,Richard J. Isherwood,0,4,...,0,1,0,0,0,0,0,3,0,10
4,ANN279301,ANN2,1979,3,1,N Face-W Ridge,UK,Paul Moores,0,4,...,0,1,0,0,0,0,0,4,0,10


In [500]:
xpd_peak = pd.merge(xpd, peaks, on='peakid')
xpd_peak.shape
#xpd_peak.loc[2]
xpd_peak['trekyear'].value_counts(dropna=False)

(9959, 43)

NaN       9798
1978.0      91
2002.0      68
20.0         2
Name: trekyear, dtype: int64

In [710]:
memb = pd.read_csv('members.csv')
memb.shape

(65534, 85)

In [711]:
123456789112345678911234567891123456789112345678911234567891123456789112345678981234567891123456789112345678911234567891
123456789112345678911234567891123456789112345678911234567891123456789112

123456789112345678911234567891123456789112345678911234567891123456789112345678981234567891123456789112345678911234567891

123456789112345678911234567891123456789112345678911234567891123456789112